In [1]:
%pwd

'd:\\ML\\LiveProject\\ML-CreditCardDefaulter\\research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\ML\\LiveProject\\ML-CreditCardDefaulter'

In [27]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path
    file_name:str

In [7]:
from ml_creditcard_defaulter.constants import *
from ml_creditcard_defaulter.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir,
                file_name=config.file_name
            )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from ml_creditcard_defaulter import logger
from ml_creditcard_defaulter.utils.common import get_size

In [10]:
import pandas as pd

In [30]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def extract_data(self):
        unzip_path = self.config.unzip_dir
        destination_path = os.path.join(unzip_path, self.config.file_name)
        os.makedirs(unzip_path, exist_ok=True)

        data = pd.read_excel(self.config.source_URL)
        data.to_csv(destination_path, index=False)
        logger.info(f"File successfully moved from {self.config.source_URL} to {destination_path}")
  

In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_data()
except Exception as e:
    raise e

[2025-01-24 11:58:30,247: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-24 11:58:30,249: INFO: common: yaml file: params.yaml loaded successfully]


[2025-01-24 11:58:30,251: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-24 11:58:30,253: INFO: common: created directory at: artifacts]
[2025-01-24 11:58:30,255: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-24 11:58:32,700: INFO: 1632102995: File successfully moved from data/default of credit card clients.xls to artifacts/data_ingestion]
